<a href="https://colab.research.google.com/github/Katinon/Self-Study/blob/main/%ED%98%BC%EC%9E%90-%EA%B3%B5%EB%B6%80%ED%95%98%EB%8A%94-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%94%A5%EB%9F%AC%EB%8B%9D/11_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 앙상블 학습
# 결정트리 기반의 정형 데이터를 다루는 뛰어난 알고리즘

# 랜덤 포레스트
# 앙상블 학습으로, 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 부트스트랩 방식 사용
# 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 과대적합을 막고 테스트에 안정적인 성능

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("https://bit.ly/wine_csv_data")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
# 훈련세트, 테스트세트 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)

In [11]:
from sklearn.model_selection import cross_validate #교차 검증
from sklearn.ensemble import RandomForestClassifier #랜덤포레스트 분류모델
# 랜덤포레스트 객체 생성
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# return_train_score로 훈련세트 점수도 같이 반환, 기본값 false, 5폴드 교차검증
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [12]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 도수, 당도, pH

# 결정 트리에서보다 당도의 중요도가 떨어지고, 다른 특성의 중요도 상승
# 특성의 일부를 랜덤하게 선택하여 훈련하기 때문
# 과대 적합을 줄이고 일반화 성능을 높임

[0.23167441 0.50039841 0.26792718]


In [15]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB라고 부름 (Out Of Bag)
# 이것을 사용하여 부트스트랩 샘플로 훈련한 결정트리 평가 가능, 검증 세트의 역할
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)
rf.fit(train_input, train_target)
rf.oob_score_ # 교차 검증으로 얻은 점수와 비슷함

0.8934000384837406

In [16]:
# 엑스트라 트리
# 부트스트랩 샘플이 아닌 전체 샘플 사용
# 노드를 분할할 때 가장 좋은 분할이 아닌 무작위로 분할 (splitter = random)
# 성능은 낮아지지만 많은 트리를 앙상블하기 때문에
# 과대적합을 막고 검증세트의 점수를 높이는 효과가 있음
# 더 많은 결정트리를 훈련해야 하지만 랜덤하게 노드를 분할하여 빠른 계산속도

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [18]:
et.fit(train_input, train_target)
print(et.feature_importances_)
# 도수, 당도, pH
# 마찬가지로 당도에 대한 의존성이 결정 트리보다 작음
# 엑스트라 트리 회귀는 ExtraTreesRegressor클래스

[0.20183568 0.52242907 0.27573525]


In [21]:
# 그레이디언트 부스팅
# 깊이가 얇은 결정 트리를 이용하여 이전 트리의 오차 보완
# 기본적으로 깊이 3의 결정트리 100개 사용
# 과대적합에 강하고, 일반화 성능 높음
# 경사하강법 사용, 분류에서는 로지스틱 손실함수, 회귀에서는 평균 제곱 오차함수 사용

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1) # 훈련세트, 교차검증
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [25]:
# 학습률을 증가시키고 트리의 개수를 늘리면 성능 향상
# 결정트리개수의 기본값은 100, 학습률은 0.1
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2 ,random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [26]:
# 그래이디언트 부스팅이 랜덤포레스트보다 일부 특성에 더 집중
gb.fit(train_input, train_target)
print(gb.feature_importances_)

# subsample 매개변수는 트리 훈련에 사용할 훈련 세트의 비율을 정하는데
# 기본값은 1.0으로, 전체 훈련세트를 사용하지만
# 1보다 작으면 훈련세트의 일부를 사용하고, 이는 확률적 경사하강법이나 미니배치 하강법과 비슷하다

# 그래이디언트 부스팅이 랜덤포레스트보다 성능은 조금 더 좋으나 훈련 속도가 느림
# 회귀 버전은 GradientBoostingRegressor 클래스

[0.15872278 0.68010884 0.16116839]


In [29]:
# 히스토그램 기반 그레이디언트 부스팅
# 그레이디언트 부스팅의 성능을 더욱 개선한 버전
# 입력 특성을 256개의 구간으로 나누어 노드 분할 시 최적의분할을 빠르게 찾을 수 있음
# 누락된 값은 256개의 구간 중 하나를 사용, 누락된 특성이 있어도 전처리할 필요 없음

# 성능을 높이려면 max_iter 매개변수 이용

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 그레이디언트 부스팅보다 조금 더 높은 성능 제공

0.9321723946453317 0.8801241948619236


In [33]:
# 특성 중요도 계산은 permutation_importance()
# 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 특성 중요도 계산
# n_repeats는 랜덤하게 섞을 횟수 지정, 기본값 5

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
#특성 중요도, 평균, 표준편차 반환

result1= permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result1.importances_mean)

result2= permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result2.importances_mean)
# 도수, 당도, pH
# 모델을 실전에 투입할 때 어떤 특성에 관심둘지 예상가능

[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


In [34]:
hgb.score(test_input, test_target)
# 87% 정도의 정확도

0.8723076923076923

In [37]:
# 히스토그램 기반 그래이디언트 부스팅 알고리즘
# 을 구현한 다른 라이브러리 중
# XGBoost
# 사이킷런의 cross_validate와 같이 사용 가능

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [40]:
# LightGBM 또한 히스토그램 기반 그래이디언트 부스팅
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
